## Imports

In [5]:
from glob import glob 
from PIL import Image
import cv2
import numpy as np
from PIL import Image
import pytesseract

In [ ]:

# rgb_color = tuple(int(CHAT_BUBBLE_COLOR[i:i+2], 16) for i in (1, 3, 5))  # (42, 42, 45)

# Color of receiving chat in apple devices:
CHAT_BUBBLE_COLOR = '#2a2a2d'
CHAT_TEXT_COLOR = '#ffffff'

# Loading Images
image_dir = "../data/raw_images/"
image_list = sorted(glob(image_dir + "*.PNG"))

sample_image = image_list[0]

image = cv2.imread(sample_image)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


# Convert image to HSV for better color masking
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Convert target RGB to HSV
target = np.uint8([[rgb_color]])
target_hsv = cv2.cvtColor(target, cv2.COLOR_RGB2HSV)[0][0]


# Define color range around the target (tune these for better results)
tolerance = 20  # how much to tolerate variation in color
lower = np.array([max(target_hsv[0]-tolerance, 0), 30, 30])
upper = np.array([min(target_hsv[0]+tolerance, 179), 255, 255])


# Create mask
mask = cv2.inRange(hsv, lower, upper)

# Optional: clean up mask
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

extracted_texts = []

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w * h > 500:  # filter out tiny areas
        bubble_crop = image_rgb[y:y+h, x:x+w]
        
        # Convert to PIL image
        pil_img = Image.fromarray(bubble_crop)

        # OCR
        text = pytesseract.image_to_string(pil_img)
        if text.strip():
            extracted_texts.append(text.strip())


In [25]:

# Output all extracted bubble texts
for i, text in enumerate(extracted_texts, 1):
    print(f"Bubble {i}: {text}")


Bubble 1: @ © @ -»‘s'
Bubble 2: 4a

Melissa was there when you
opened the garage door to the
old house and | was moving us to
the new one and you said “God

_ bless him”
Bubble 3: | ‘of
| m on stage 3/4 of grief
Bubble 4: Give katie and caroline hugs
_ from me please
Bubble 5: 4a

she’s not mad - she's like what
the hell - you could see
_ EVERYTHING from up here!? y

>»
Bubble 6: 4 >

_ Love you too mom 1
Bubble 7: ot) «LTE @


In [ ]:

# Comparison to ground truth
Image.open(sample_image)


